In [21]:
import pandas as pd

In [22]:
teams = pd.read_csv('MTeams.csv')
seeds = pd.read_csv('MNCAATourneySeeds.csv')
slots = pd.read_csv('MNCAATourneySlots.csv')
results = pd.read_csv('MNCAATourneyCompactResults.csv')

In [23]:
results['WTeamID'] = results['WTeamID'].astype(str)
results['LTeamID'] = results['LTeamID'].astype(str)
results['Season'] = results['Season'].astype(str)
slots['Season'] = slots['Season'].astype(str)
seeds['Season'] = seeds['Season'].astype(str)

In [24]:
seeds.head()

,Season,Seed,TeamID
0,1985,W01,1207
1,1985,W02,1210
2,1985,W03,1228
3,1985,W04,1260
4,1985,W05,1374


In [25]:
slots['Round'] = slots.Slot.apply(lambda x: x[:2] if x[0]=='R' else 'R0')
slots.head()

,Season,Slot,StrongSeed,WeakSeed,Round
0,1985,R1W1,W01,W16,R1
1,1985,R1W2,W02,W15,R1
2,1985,R1W3,W03,W14,R1
3,1985,R1W4,W04,W13,R1
4,1985,R1W5,W05,W12,R1


In [26]:
rounds = {134: 'R0', 135: 'R0', 136: 'R1', 137: 'R1', 138: 'R2', 139: 'R2', 140: 'R2', 143: 'R3', 144: 'R3', 145: 'R4', 146: 'R4', 147: 'R4', 148: 'R4', 152: 'R5', 154: 'R6'}

In [27]:
results['Round'] = results.DayNum.map(rounds)
results

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,Round
0,1985,136,1116,63,1234,54,N,0,R1
1,1985,136,1120,59,1345,58,N,0,R1
2,1985,136,1207,68,1250,43,N,0,R1
3,1985,136,1229,58,1425,55,N,0,R1
4,1985,136,1242,49,1325,38,N,0,R1
...,...,...,...,...,...,...,...,...,...
2312,2021,148,1211,85,1425,66,N,0,R4
2313,2021,148,1417,51,1276,49,N,0,R4
2314,2021,152,1124,78,1222,59,N,0,R5
2315,2021,152,1211,93,1417,90,N,1,R5


In [28]:
from tqdm import tqdm

In [39]:
import math
def probability(r1, r2):
    return 1.0*1.0/(1+1.0*math.pow(10,1.0*(r1-r2)/400))
def elo_rating(Ra, Rb, d, K=20):
    Pb = probability(Ra, Rb)
    Pa = probability(Rb, Ra)
    Ra = Ra + K * (d - Pa)
    Rb = Rb + K * (1 - d - Pb)
    return Ra, Rb

In [40]:
dfs = []
K = 20
team_ratings = {str(t): 800 for t in teams.TeamID.unique()}
for s in slots.Season.unique()[:-5]:
    tmp_seeds = seeds[seeds.Season==s].copy()
    seed_dict = {r.Seed: str(r.TeamID) for i, r in tmp_seeds.iterrows()}
    for r in ['R'+str(i) for i in range(7)]:
        try:
            tmp = slots[(slots.Season==s) & (slots.Round==r)].copy()
            tmp['StrongTeamID'] = tmp.StrongSeed.map(seed_dict)
            tmp['WeakTeamID'] = tmp.WeakSeed.map(seed_dict)
            tmp_results = results[(results.Season==s) & (results.Round==r)].copy()
            wteams = [v for v in tmp_results['WTeamID'].values]
            tmp['Result'] = tmp.apply(lambda r: 1 if r.StrongTeamID in wteams else 0, axis=1)
            tmp['WTeamID'] = tmp.apply(lambda r: r.StrongTeamID if r.StrongTeamID in wteams else r.WeakTeamID, axis=1)
            new_dict = {r.Slot: r.WTeamID for i, r in tmp.iterrows()}
            seed_dict.update(new_dict)
            df = tmp.merge(tmp_results, how='left', on=['Season','Round','WTeamID'])
            for i, x in df.iterrows():
                Ra = team_ratings[x.StrongTeamID]
                Rb = team_ratings[x.WeakTeamID]
                team_ratings[x.StrongTeamID], team_ratings[x.WeakTeamID] = elo_rating(Ra, Rb, x.Result, K)
            dfs.append(df)
        except Exception as e:
            pass

final = pd.concat(dfs)

In [41]:
max(team_ratings, key=team_ratings.get)

'1246'

In [42]:
probability(team_ratings['1192'], team_ratings['1195'])

0.5728786860833796

In [43]:
sub = []
team_ratings_new = team_ratings.copy()
for s in slots.Season.unique()[-5:]:
    tmp_seeds = seeds[seeds.Season==s].copy()
    teams_ = sorted(tmp_seeds.TeamID.tolist()).copy()
    a = {}
    for i in range(len(teams_)):
        for j in range(i+1, len(teams_)):
            sub.append({'ID': str(s)+'_'+str(teams_[i])+'_'+str(teams_[j]), 'Pred': probability(team_ratings[str(teams_[j])], team_ratings[str(teams_[i])])})
pd.DataFrame(sub).to_csv('submission.csv', index=False)